In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning);

import sys
sys.executable

'/home/mingrui/anaconda3/envs/py36_cu90/bin/python'

In [2]:
import os
import glob
import shutil
from random import shuffle

# plotting
import PIL
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import Image
plt.rcParams['figure.figsize'] = [15, 15]

import deephistopath.wsi.slide as slide

import py_wsi
import py_wsi.imagepy_toolkit as tk

from wsi_utils import multiprocessing_pipeline, file_stats, turtle_stats

import pandas as pd
pd.set_option('display.max_columns',None)

# Setup multi classes

# Folder structure

/media/disk2/PathologyNet/  
|-- TCGA_Glioma  
|-- Meningiomas  
|-- TCGA_Lung  
|-- ...  


In [3]:
tumor_folder_names = [['TCGA_Glioma'], ['TCGA_Lung'], ['TCGA_Kidney'], 
                      ['TCGA_Breast'], ['TCGA_Uterus'], ['TCGA_Colorectal'],
                      ['TCGA_Bladder'], ['TCGA_Thyroid']]
tumor_type_list = []

SYNOLOGY_DIR = '/media/pathimg/'
PATH_NET_DIR = '/media/disk2/PathologyNet'
#PATH_NET_DIR = './data/'

for tumor in tumor_folder_names:
    # use first as origin folder name
    tumor_dir = os.path.join(SYNOLOGY_DIR, tumor[0])
    # use last as destination folder name
    db_dir = os.path.join(PATH_NET_DIR, tumor[-1])
    if os.path.exists(db_dir):
        shutil.rmtree(db_dir)
    os.mkdir(db_dir)
    # name db with destination folder name
    db_name = tumor[-1] + '_patch_db'
    tumor_type_list.append([tumor_dir, db_dir, db_name])
    
print(tumor_type_list)

[['/media/pathimg/TCGA_Glioma', '/media/disk2/PathologyNet/TCGA_Glioma', 'TCGA_Glioma_patch_db'], ['/media/pathimg/TCGA_Lung', '/media/disk2/PathologyNet/TCGA_Lung', 'TCGA_Lung_patch_db'], ['/media/pathimg/TCGA_Kidney', '/media/disk2/PathologyNet/TCGA_Kidney', 'TCGA_Kidney_patch_db'], ['/media/pathimg/TCGA_Breast', '/media/disk2/PathologyNet/TCGA_Breast', 'TCGA_Breast_patch_db'], ['/media/pathimg/TCGA_Uterus', '/media/disk2/PathologyNet/TCGA_Uterus', 'TCGA_Uterus_patch_db'], ['/media/pathimg/TCGA_Colorectal', '/media/disk2/PathologyNet/TCGA_Colorectal', 'TCGA_Colorectal_patch_db'], ['/media/pathimg/TCGA_Bladder', '/media/disk2/PathologyNet/TCGA_Bladder', 'TCGA_Bladder_patch_db'], ['/media/pathimg/TCGA_Thyroid', '/media/disk2/PathologyNet/TCGA_Thyroid', 'TCGA_Thyroid_patch_db']]


In [4]:
def process_tumor_type(tumor_dir, db_dir, db_name, num_to_process):
    num_to_process -= 1
    turtle = py_wsi.Turtle(tumor_dir, db_dir, db_name)
    turtle_stats(turtle)
#     file_stats(turtle.files_path[0])
    files_path = turtle.files_path[:num_to_process]
    files_name = turtle.files[:num_to_process]
    
    return files_path, files_name

In [5]:
for tumor_type in tumor_type_list:
    print('\nExplore', tumor_type[0])
    process_tumor_type(tumor_type[0], tumor_type[1], tumor_type[2], 28)


Explore /media/pathimg/TCGA_Glioma
1704 WSI found in directory.


Total WSI images: 1704
LMDB name: TCGA_Glioma_patch_db
file names: ['TCGA-06-0171-01Z-00-DX3.svs' 'TCGA-R8-A73M-01Z-00-DX1.svs'
 'TCGA-QH-A6X8-01Z-00-DX1.svs' 'TCGA-QH-A65X-01Z-00-DX1.svs'
 'TCGA-02-0009-01Z-00-DX3.svs']
file paths: ['/media/pathimg/TCGA_Glioma/TCGA-06-0171-01Z-00-DX3.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-R8-A73M-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-QH-A6X8-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-QH-A65X-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-02-0009-01Z-00-DX3.svs']

Explore /media/pathimg/TCGA_Lung
1053 WSI found in directory.


Total WSI images: 1053
LMDB name: TCGA_Lung_patch_db
file names: ['TCGA-39-5028-01Z-00-DX1.7994ec22-746d-4c30-8138-e6c9bc67c71f.svs'
 'TCGA-55-8508-01Z-00-DX1.96B173E2-A298-4017-9AEC-113E547A3272.svs'
 'TCGA-55-8203-01Z-00-DX1.f530d261-be19-4ff0-98d0-27b789dffb23.svs'
 'TCGA-64-5815-01Z-00-DX1.1E3600B5-7E83-47BE-AC70-E0C2D623740B.svs'
 'TCGA-

In [6]:
# %%capture capt
for tumor_type in tumor_type_list:
    files_path, files_name = process_tumor_type(tumor_type[0], tumor_type[1], tumor_type[2], 100)
    db_dir = tumor_type[1]
    # create db dir if doesn't exist at destination
    if os.path.exists(db_dir):
        shutil.rmtree(db_dir)
    os.mkdir(db_dir)
    multiprocessing_pipeline(tumor_type[0], db_dir, files_path, files_name, tile_size=512, objective_power=20, num_tiles=50)

1704 WSI found in directory.


Total WSI images: 1704
LMDB name: TCGA_Glioma_patch_db
file names: ['TCGA-06-0171-01Z-00-DX3.svs' 'TCGA-R8-A73M-01Z-00-DX1.svs'
 'TCGA-QH-A6X8-01Z-00-DX1.svs' 'TCGA-QH-A65X-01Z-00-DX1.svs'
 'TCGA-02-0009-01Z-00-DX3.svs']
file paths: ['/media/pathimg/TCGA_Glioma/TCGA-06-0171-01Z-00-DX3.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-R8-A73M-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-QH-A6X8-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-QH-A65X-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-02-0009-01Z-00-DX3.svs']
================START================
/media/pathimg/TCGA_Glioma
Number of processes: 56
Number of training images: 99
Time elapsed: 0:00:14.911675
Applying filters to images (multiprocess)

Processing slide #1
Processing slide #8
Processing slide #6
Processing slide #4
Processing slide #2
Processing slide #9
Processing slide #11
Processing slide #15
Processing slide #16
Processing slide #18
Processing slide #13
Processing slide #22
Processi

# Move files to training folders

In [7]:
def copy_tiles(slides, dest, start, end):
    print('copy to', dest)
    for slide in slides[start:end]:
        tiles = glob.glob(os.path.join(slide, '*'))
        for tile in tiles:
            shutil.copy(tile, dest)

In [10]:
DATA_DIR = 'data'
train_dir = os.path.join(DATA_DIR, 'train')
val_dir = os.path.join(DATA_DIR, 'val')
test_dir = os.path.join(DATA_DIR, 'test')

for tumor_type in tumor_type_list:
    tumor_name = tumor_type[2][:-9]
    tumor_db_dir = tumor_type[1]

    tumor_tile_dir = os.path.join(tumor_db_dir, 'tiles_png')

    train_tumor_dir = os.path.join(train_dir, tumor_name)
    val_tumor_dir = os.path.join(val_dir, tumor_name)
    test_tumor_dir = os.path.join(test_dir, tumor_name)

    if not os.path.exists(train_tumor_dir):
        os.mkdir(train_tumor_dir)

    if not os.path.exists(val_tumor_dir):
        os.mkdir(val_tumor_dir)
    
    if not os.path.exists(test_tumor_dir):
        os.mkdir(test_tumor_dir)

    all_tumor_slides = glob.glob(os.path.join(tumor_tile_dir, '*'), recursive=True)
    
    print(tumor_name, len(all_tumor_slides), 'slides')
    
    shuffle(all_tumor_slides)
    copy_tiles(all_tumor_slides, train_tumor_dir, 0, 70)
    copy_tiles(all_tumor_slides, val_tumor_dir, 70, 85)
    copy_tiles(all_tumor_slides, test_tumor_dir, 85, 99)

TCGA_Glioma 99 slides
copy to data/train/TCGA_Glioma
copy to data/val/TCGA_Glioma
copy to data/test/TCGA_Glioma
TCGA_Lung 99 slides
copy to data/train/TCGA_Lung
copy to data/val/TCGA_Lung
copy to data/test/TCGA_Lung
TCGA_Kidney 99 slides
copy to data/train/TCGA_Kidney
copy to data/val/TCGA_Kidney
copy to data/test/TCGA_Kidney
TCGA_Breast 99 slides
copy to data/train/TCGA_Breast
copy to data/val/TCGA_Breast
copy to data/test/TCGA_Breast
TCGA_Uterus 99 slides
copy to data/train/TCGA_Uterus
copy to data/val/TCGA_Uterus
copy to data/test/TCGA_Uterus
TCGA_Colorectal 99 slides
copy to data/train/TCGA_Colorectal
copy to data/val/TCGA_Colorectal
copy to data/test/TCGA_Colorectal
TCGA_Bladder 99 slides
copy to data/train/TCGA_Bladder
copy to data/val/TCGA_Bladder
copy to data/test/TCGA_Bladder
TCGA_Thyroid 99 slides
copy to data/train/TCGA_Thyroid
copy to data/val/TCGA_Thyroid
copy to data/test/TCGA_Thyroid
